In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import pandas as pd
import scipy
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture

plt.close('all')

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')



In [ ]:
# resultsTimes = ["temp"]
# resultsTimes = ["20240406-224437", "20240406-234343", "20240406-235655"]
# resultsTimes = ["20240408-235543", "20240409-005549", "20240409-005827"]
# resultsTimes = ["20240409-203318"]
resultsTimes = ["20240420-235844", "20240421-002223", "20240421-003940", "20240421-005249", "20240421-013347", "20240421-015347"]
outputs = []
inputs = []
for resultsTime in resultsTimes:
    df = pd.read_csv(f'{home_dir}/resultsMC/outputs_{resultsTime}.csv', header=[0, 1], index_col=0)
    outputs.append(df)
    df = pd.read_csv(f'{home_dir}/resultsMC/inputs_{resultsTime}.csv', index_col=0)
    df["group"] = resultsTime
    inputs.append(df)
outputs = pd.concat(outputs, axis = "index", ignore_index=True)
inputs = pd.concat(inputs, axis = "index", ignore_index=True)
display(inputs, outputs)


In [ ]:
dfFull = pd.DataFrame()
ventHeaders = outputs.columns.get_level_values(0).unique().values
for header in ventHeaders:
    df = pd.concat([inputs, outputs[header][:]], axis=1)
    df["run"] = df.index.values
    dfFull = pd.concat([dfFull, df], axis = "index")

dfFull = dfFull.dropna()
dfFull = dfFull.reset_index(drop=True)
dfFull

In [ ]:
dropCondition = dfFull["ceilingMinusFloor"] > 10
dfFull = dfFull[~dropCondition]

In [ ]:
dfFull = dfFull[dfFull["hVent"] == 23]

In [ ]:
def ventRi(delT, V, H = 3):
    g = 10
    Tref = 288.15
    return g * delT / Tref * H / V**2

dfFull.loc[:, "Ri"] = ventRi(dfFull["ceilingMinusFloor"], dfFull["windSpeed"])
dfFull.loc[:, "logRi"] = np.log(dfFull["Ri"])
dfFull

In [ ]:
qois = ["outMinusFloor", "ceilingMinusFloor", "extWallMinusFloor", "intWallMinusFloor"]
fig = px.histogram(
    dfFull,
    x = qois,
    marginal="box", # or violin, rug
    barmode = "group"
    )
fig.show()

fig = px.line(dfFull, y = qois)
fig.show()


In [ ]:
facet_col_order = ['Light', 'Medium', 'Heavy']
fig = px.scatter(dfFull, x="floorTempAdjustment", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="hInterior", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="alphaRoof", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="dVent", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="nVent", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="windSpeed", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="wallRoughness", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="hExterior", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="ToutMinusTint", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="maxToutVent", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order}, hover_data=["nVent", "run"])
fig.show()
# fig = px.scatter(dfFull, x="logRi", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
# fig.show()

In [ ]:
fig = px.scatter(dfFull, x="windSpeed", y="logRi", symbol="material_type")
fig.show()
fig = px.scatter(dfFull.loc[dfFull["windSpeed"] >= 2], x="windSpeed", y="Ri", symbol="material_type")
fig.show()

In [ ]:
fig = px.scatter_3d(dfFull, x="outMinusFloor", y="ceilingMinusFloor", z="intWallMinusFloor", color="extWallMinusFloor", symbol="material_type", size = "windSpeed")
fig.show()

fig = px.scatter_3d(dfFull, x="outMinusFloor", y="ceilingMinusFloor", z="intWallMinusFloor", color="nVent", symbol="material_type", size = "windSpeed")
fig.show()

# fig = px.scatter(dfFull, x="outMinusFloor", y="ceilingMinusFloor", color="intWallMinusFloor", symbol="material_type",  size = "windSpeed")
# fig.show()
# fig = px.scatter(dfFull, x="outMinusFloor", y="extWallMinusFloor", color="intWallMinusFloor", symbol="material_type",  size = "windSpeed")
# fig.show()
# fig = px.scatter(dfFull, x="ceilingMinusFloor", y="extWallMinusFloor", color="intWallMinusFloor", symbol="material_type",  size = "windSpeed")
# fig.show()

In [ ]:
plt.figure()
qoiX = dfFull[qois]
U, S, V = np.linalg.svd(qoiX)
plt.plot(S)
display(S)

In [ ]:
qoiX @ V[0:2,:].T

In [ ]:
dfFull['windSpeed']

In [ ]:
fig = px.scatter_3d(qoiX @ V.T, x=0, y=1, z=2, color=3)
fig.show()

fig = px.scatter(qoiX @ V[0:2,:].T, x=0, y=1, color=dfFull["nVent"], symbol=dfFull["material_type"])
fig.show()

fig = px.scatter(qoiX @ V[2:,:].T, x=0, y=1, color=dfFull["nVent"], symbol=dfFull["material_type"])
fig.show()

In [ ]:
# Calculate mean and standard deviation for each column
mean = np.mean(qoiX, axis=0)
std = np.std(qoiX, axis=0)

# Perform z-score normalization
qoiXNorm = (qoiX - mean) / std
qoiXNorm

In [ ]:
# Assuming X contains your 4-dimensional data
# X = [[feature1_1, feature2_1, feature3_1, feature4_1],
#      [feature1_2, feature2_2, feature3_2, feature4_2],
#      ...
#      [feature1_n, feature2_n, feature3_n, feature4_n]]

# Step 2: Choose the number of clusters (k)
k = 3

# Step 3: Apply K-Means Algorithm
model = KMeans(n_clusters=k, random_state=0, n_init='auto')
# model = GaussianMixture(n_components=k)  # Number of clusters
# model = AgglomerativeClustering(n_clusters=k)
# eps = 1  # Maximum distance between two samples to be considered as neighbors
# min_samples = 4  # Minimum number of samples in a neighborhood for a point to be considered as a core point
# model = DBSCAN(eps=eps, min_samples=min_samples)
# model = SpectralClustering(n_clusters=k)

# Step 4: Fit the model
model.fit(qoiXNorm)

# Step 5: Cluster Assignment
labels = model.labels_

# Step 6: Interpret Results
centroids = model.cluster_centers_

# Analyze the clusters and visualize the results as needed


# Plotting the clusters
plt.figure(figsize=(10, 6))

# Plot data points
sns.scatterplot(data=qoiX, x=qois[0], y=qois[1], hue=labels, style=dfFull["material_type"], palette='viridis', legend='full', s=50)
# Plot centroids
plt.scatter(centroids[:, 0], centroids[:, 1], marker='x', s=100, c='red', label='Centroids')
plt.legend()

plt.figure(figsize=(10, 6))
# Plot data points
sns.scatterplot(data=qoiX, x=qois[2], y=qois[3], hue=labels, style=dfFull["material_type"], palette='viridis', legend='full', s=50)
# Plot centroids
plt.scatter(centroids[:, 2], centroids[:, 3], marker='x', s=100, c='red', label='Centroids')
plt.legend()

fig = px.histogram(
    dfFull,
    x = "windSpeed",
    facet_row = "material_type",
    facet_col = "nVent",
    color = labels,
    # marginal="box", # or violin, rug
    barmode = "group",
    )

fig.update_layout(width=1400, height=800)  # Set width and height as per your preference
fig.show()

In [ ]:
qoiXNormReduced = qoiXNorm.iloc[:,0:2]
# Assuming X contains your 4-dimensional data
# X = [[feature1_1, feature2_1, feature3_1, feature4_1],
#      [feature1_2, feature2_2, feature3_2, feature4_2],
#      ...
#      [feature1_n, feature2_n, feature3_n, feature4_n]]

# Step 2: Choose the number of clusters (k)
k = 3

# Step 3: Apply K-Means Algorithm
model = KMeans(n_clusters=k, random_state=0, n_init='auto')
# model = GaussianMixture(n_components=k)  # Number of clusters
# model = AgglomerativeClustering(n_clusters=k)
# eps = 1  # Maximum distance between two samples to be considered as neighbors
# min_samples = 4  # Minimum number of samples in a neighborhood for a point to be considered as a core point
# model = DBSCAN(eps=eps, min_samples=min_samples)
# model = SpectralClustering(n_clusters=k)

# Step 4: Fit the model
model.fit(qoiXNormReduced)

# Step 5: Cluster Assignment
labels = model.labels_

# Step 6: Interpret Results
centroids = model.cluster_centers_

# Analyze the clusters and visualize the results as needed


# Plotting the clusters
plt.figure(figsize=(10, 6))

# Plot data points
sns.scatterplot(data=qoiXNormReduced, x=qois[0], y=qois[1], hue=labels, style=dfFull["material_type"], palette='viridis', legend='full', s=50)
# Plot centroids
plt.scatter(centroids[:, 0], centroids[:, 1], marker='x', s=100, c='red', label='Centroids')
plt.legend()

# plt.figure(figsize=(10, 6))
# # Plot data points
# sns.scatterplot(data=qoiXNormReduced, x=qois[2], y=qois[3], hue=labels, style=dfFull["material_type"], palette='viridis', legend='full', s=50)
# # Plot centroids
# plt.scatter(centroids[:, 2], centroids[:, 3], marker='x', s=100, c='red', label='Centroids')
# plt.legend()

fig = px.histogram(
    dfFull,
    x = "windSpeed",
    facet_row = "material_type",
    facet_col = "nVent",
    color = labels,
    # marginal="box", # or violin, rug
    barmode = "group",
    )

fig.update_layout(width=1400, height=800)  # Set width and height as per your preference
fig.show()